# Compare two GEOS-Chem Classic datasets

## Overview of this Notebook

* Import dependencies
* Define functions: utility, grid, and plotting
* Define data directories and global variables
* Compare restart files
* Store areas for normalization
* Compare SpeciesConc diagnostic collection
* Create PDFs for subset of species
* Other diagnostic collections

## Import dependencies

In [1]:
import os
import numpy as np
import xarray as xr
import cubedsphere as cs
import xesmf as xe
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import ticker
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
from cartopy import crs
from cartopy.mpl.geoaxes import GeoAxes

%matplotlib inline
import warnings; warnings.filterwarnings("ignore")

# Also define colormap. Colormap file source: https://bitbucket.org/gcst/gcpy
rgb_WhGrYlRd = np.genfromtxt('/n/home08/elundgren/GC/python/WhGrYlRd.txt',delimiter=' ')
WhGrYlRd = ListedColormap(rgb_WhGrYlRd/255.0)

## Define general utility functions

In [2]:
def get_gcc_filepath(outputdir, collection, day, time):
    if collection == 'Emissions':
        filepath = os.path.join(outputdir, 'HEMCO_diagnostics.{}{}.nc'.format(day,time))
    else:
        filepath = os.path.join(outputdir, 'GEOSChem.{}.{}_{}z.nc4'.format(collection,day,time))
    return filepath

In [3]:
def check_paths( refpath, devpath):
    if not os.path.exists(refpath):
        print('ERROR! Path 1 does not exist: {}'.format(refpath))
    else:
        print('Path 1 exists: {}'.format(refpath))
    if not os.path.exists(devpath):
        print('ERROR! Path 2 does not exist: {}'.format(devpath))
    else:
        print('Path 2 exists: {}'.format(devpath))

In [4]:
def compare_varnames(refdata, refstr, devdata, devstr):
    
    # Find common variables in collection by generating lists and list overlap
    refvars = [k for k in refdata.data_vars.keys()]
    devvars= [k for k in devdata.data_vars.keys()]
    commonvars = sorted(list(set(refvars).intersection(set(devvars))))
    refonly = [v for v in refvars if v not in devvars]
    devonly = [v for v in devvars if v not in refvars]
    dimmismatch = [v for v in commonvars if refdata[v].ndim != devdata[v].ndim]
    commonvars2D = [v for v in commonvars if refdata[v].ndim == 3]
    commonvars3D = [v for v in commonvars if devdata[v].ndim == 4]
    
    # Print information on common and mismatching variables, as well as dimensions
    print('{} common variables ({} are 2-dim and {} are 3-dim)'.format(len(commonvars), len(commonvars2D), len(commonvars3D)))
    if len(refonly) > 0:
        print('{} variables in {} only (skip)'.format(len(refnly)),refstr)
        print('   Variable names: {}'.format(refonly))
    if len(devonly) > 0:
        print('{} variables in {} only (skip)'.format(len(devonly)),devstr)
        print('   Variable names: {}'.format(devonly))
    if len(dimmismatch) > 0:
        print('{} common variables have different dimensions'.format(len(dimmismatch)))
        print('   Variable names: {}'.format(dimmismatch))
        
    return [commonvars, commonvars2D, commonvars3D]

In [5]:
def get_stats(refdata, refstr, devdata, devstr, varname):
    refvar = refdata[varname]
    devvar = devdata[varname]
    units = refdata[varname].units
    print('Data units: {}'.format(units))
    print('Array sizes:')
    print('    {}:  {}'.format(refstr,refvar.shape))
    print('    {}:  {}'.format(devstr,devvar.shape))
    print('Global stats:')
    print('  Mean:')
    print('    {}:  {}'.format(refstr,np.round(refvar.values.mean(),20)))
    print('    {}:  {}'.format(devstr,np.round(devvar.values.mean(),20)))
    print('  Min:')
    print('    {}:  {}'.format(refstr,np.round(refvar.values.min(),20)))
    print('    {}:  {}'.format(devstr,np.round(devvar.values.min(),20)))
    print('  Max:')
    print('    {}:  {}'.format(refstr,np.round(refvar.values.max(),20)))
    print('    {}:  {}'.format(devstr,np.round(devvar.values.max(),20)))
    print('  Sum:')
    print('    {}:  {}'.format(refstr,np.round(refvar.values.sum(),20)))
    print('    {}:  {}'.format(devstr,np.round(devvar.values.sum(),20)))

In [6]:
def get_collection_data(refdir, devdir, collection, day, time):
    reffile = get_gcc_filepath(refdir, collection, day, time)
    devfile = get_gcc_filepath(devdir, collection, day, time)
    check_paths(reffile, devfile)
    refdata = xr.open_dataset(reffile)
    devdata = xr.open_dataset(devfile)
    return [refdata, devdata]

## Define grid functions

In [7]:
def make_grid_LL(llres):
    [dlat,dlon] = list(map(float, llres.split('x')))
    lon_b = np.linspace(-180 - dlon/2, 180 - dlon/2, int(360/dlon) + 1, endpoint=True)
    lat_b = np.linspace(-90 - dlat/2, 90 + dlat/2, int(180/dlat) + 2, endpoint=True).clip(-90,90)
    lat = (lat_b[1:] + lat_b[:-1]) / 2
    lon = (lon_b[1:] + lon_b[:-1]) / 2
    llgrid = {'lat': lat, 
              'lon': lon, 
              'lat_b': lat_b, 
              'lon_b': lon_b}
    return llgrid

## Define functions for plotting

### Function to plot and compare single vertical level of GC Classic

Notes: The two GCX files must be at the same grid resolution. You can use this function to plot interactively or to generate a multi-page pdf of plots. It takes a list of variable names to plot for a single collection only. You can plot for any level and any time slice in the file. By default the colorbars for the plots will have the same range, but you can turn this feature off. Also by default the colorbar of the fractional difference between the model outputs will be limited to +/-2, but you can change this as well via the passed parameters.

In [8]:
def compare_single_level(refdata, refstr, devdata, devstr, llres, varlist=None,
                         ilev=0, itime=0, savepdf=False, pdfname='gcc_vs_gcc_map.pdf', match_cbar=True, 
                         full_ratio_range=False, normalize_by_area=False, area_ref=None, 
                         area_dev=None, check_units=True):
    
    # If no varlist is passed, plot all (surface only for 3D)
    if varlist == None:
        [varlist, commonvars2D, commonvars3D] = compare_varnames(refdata, devdata)
        print('Plotting all common variables (surface only if 3D)')
    n_var = len(varlist)

    # Get lat-lon grid
    llgrid = make_grid_LL(llres)

    # Get lat/lon extents
    [minlon, maxlon] = [min(llgrid['lon_b']), max(llgrid['lon_b'])]
    [minlat, maxlat] = [min(llgrid['lat_b']), max(llgrid['lat_b'])]
    
    # Create pdf (if saving)
    if savepdf:
        print('\nCreating {} for {} variables'.format(pdfname,n_var))
        pdf = PdfPages(pdfname)

    # Loop over variables
    for ivar in range(n_var):
        if savepdf: print('{} '.format(ivar), end='')
        varname = varlist[ivar]
        
        # Do some checks: dimensions and units
        varndim_ref = refdata[varname].ndim
        varndim_dev = devdata[varname].ndim
        if check_units: 
            assert varndim_ref == varndim_dev, 'Dimensions do not agree for {}!'.format(varname)
        units_ref = refdata[varname].units
        units_dev = devdata[varname].units
        if check_units: 
            assert units_ref == units_dev, 'Units do not match for {}!'.format(varname)
            
        # if normalizing by area, adjust units to be per m2, and adjust title string
        units = units_ref
        varndim = varndim_ref
        subtitle_extra = ''
                    
        # Slice the data
        if varndim == 4: 
            ds_ref = refdata[varname].isel(time=itime,lev=ilev)
            ds_dev = devdata[varname].isel(time=itime,lev=ilev)
        elif varndim == 3: 
            ds_ref = refdata[varname].isel(time=itime)
            ds_dev = devdata[varname].isel(time=itime)
            
        # if normalizing by area, transform on the native grid and adjust units and subtitle string
        exclude_list = ['WetLossConvFrac','Prod_','Loss_']
        if normalize_by_area and not any(s in varname for s in exclude_list):
            ds_ref.values = ds_ref.values / refdata['AREAM2'].values
            ds_dev.values = ds_dev.values / devdata['AREAM2'].values
            units = '{} m-2'.format(units)
            subtitle_extra = ', Normalized by Area'
            
        # Regrid the slices (skip for gchp vs gchp for now)
        vmin_ref = ds_ref.min()
        vmin_dev = ds_dev.min()
        vmin_cmn = np.min([vmin_ref, vmin_dev])
        vmax_ref = ds_ref.max()
        vmax_dev = ds_dev.max()
        vmax_cmn = np.max([vmax_ref, vmax_dev])
        if match_cbar: [vmin, vmax] = [vmin_cmn, vmax_cmn]
        
        # Create 2x2 figure
        figs, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, figsize=[12,9], 
                                                      subplot_kw={'projection': crs.PlateCarree()})
        # Give the figure a title
        offset = 0.96
        fontsize=25
        if varndim == 4:
            if ilev == 0: levstr = 'Surface'
            elif ilev == 22: levstr = '500 hPa'
            else: levstr = 'Level ' +  str(ilev-1)
            figs.suptitle('{}, {}'.format(varname,levstr), fontsize=fontsize, y=offset)
        elif varndim == 3: 
            figs.suptitle('{}'.format(varname), fontsize=fontsize, y=offset)
        else:
            print('varndim is 2 for {}! Must be 3 or 4.'.format(varname))   
        
        # Subplot (0,0): Ref
        ax0.coastlines()
        if not match_cbar: [vmin, vmax] = [vmin_ref, vmax_ref]
        plot0 = ax0.imshow(ds_ref, extent=(minlon, maxlon, minlat, maxlat), 
                           cmap=WhGrYlRd,vmin=vmin, vmax=vmax)
        ax0.set_title('{} (Ref){}\n{}'.format(refstr,subtitle_extra,llres)) 
        cb = plt.colorbar(plot0, ax=ax0, orientation='horizontal', pad=0.10)
        if (vmax-vmin) < 0.1 or (vmax-vmin) > 100:
            cb.locator = ticker.MaxNLocator(nbins=4)
            cb.update_ticks()
        cb.set_label(units)
        
        # Subplot (0,1): Dev
        ax1.coastlines()
        if not match_cbar: [vmin, vmax] = [vmin_dev, vmax_dev]
        plot1 = ax1.imshow(ds_dev, extent=(minlon, maxlon, minlat, maxlat), 
                           cmap=WhGrYlRd,vmin=vmin, vmax=vmax)
        ax1.set_title('{} (Dev){}\n{}'.format(devstr,subtitle_extra,llres)) 
        cb = plt.colorbar(plot1, ax=ax1, orientation='horizontal', pad=0.10)
        if (vmax-vmin) < 0.1 or (vmax-vmin) > 100:
            cb.locator = ticker.MaxNLocator(nbins=4)
            cb.update_ticks()
        cb.set_label(units)
            
        # Subplot (1,0): Difference
        absdiff = ds_dev - ds_ref
        diffabsmax = max([np.abs(absdiff.min()), np.abs(absdiff.max())])
        [vmin, vmax] = [-diffabsmax, diffabsmax]
        ax2.coastlines()
        plot2 = ax2.imshow(absdiff, extent=(minlon, maxlon, minlat, maxlat), 
                           cmap='RdBu_r',vmin=vmin, vmax=vmax)
        ax2.set_title('Difference\n(Dev - Ref)') 
        cb = plt.colorbar(plot2, ax=ax2, orientation='horizontal', pad=0.10)
        if (vmax-vmin) < 0.1 or (vmax-vmin) > 100:
            cb.locator = ticker.MaxNLocator(nbins=4)
            cb.update_ticks()
        cb.set_label(units)
        
        # Subplot (1,1): Fractional Difference (restrict to +/-2)
        fracdiff = (ds_dev - ds_ref) / ds_ref
        if full_ratio_range: 
            [vmin, vmax] = [None, None]
        else: 
            [vmin, vmax] = [-2, 2]
        ax3.coastlines()
        plot3 = ax3.imshow(fracdiff, extent=(minlon, maxlon, minlat, maxlat), 
                           cmap='RdBu_r',vmin=vmin, vmax=vmax)
        ax3.set_title('Fractional Difference\n(Dev - Ref)/Ref') 
        cb = plt.colorbar(plot3, ax=ax3, orientation='horizontal', pad=0.10)
        if (vmax-vmin) < 0.1 or (vmax-vmin) > 100:
            cb.locator = ticker.MaxNLocator(nbins=4)
            cb.update_ticks()
        cb.set_label(units)  
            
        if savepdf:    
            pdf.savefig(figs)
            plt.close(figs)
            
    if savepdf: pdf.close()

### Function to plot and compare zonal mean

Many of the same features available for plotting a single level above are also available for this function.

In [9]:
def compare_zonal_mean(refdata, refstr, devdata, devstr, llres, varlist=None, 
                       weightsdir='.', itime=0, savepdf=False, 
                       pdfname='gchp_vs_gchp_map.pdf', match_cbar=True, full_ratio_range=False, 
                       normalize_by_area=False ):

    # If no varlist is passed, plot all 3D variables in the dataset
    if varlist == None:
        [commonvars, commonvars2D, varlist] = compare_varnames(refdata, devdata)
        print('Plotting all 3D variables')
    n_var = len(varlist)
    
    # Get lat-lon grid
    llgrid = make_grid_LL(llres)
    
    # Universal plot setup
    xtick_positions = np.arange(-90,91,30)
    xticklabels = ['{}$\degree$'.format(x) for x in xtick_positions]
    ytick_positions = np.arange(0,61,20)
    yticklabels = [str(y) for y in ytick_positions]
    
    # Create pdf (if saving)
    if savepdf:
        print('\nCreating {} for {} variables'.format(pdfname, n_var))
        pdf = PdfPages(pdfname)

    # Loop over variables
    for ivar in range(n_var):
        if savepdf: print('{} '.format(ivar), end='')
        varname = varlist[ivar]
        
        # Do some checks: dimensions and units
        varndim_ref = refdata[varname].ndim
        varndim_dev = devdata[varname].ndim
        nlev = 72
        assert varndim_ref == varndim_dev, 'Dimensions do not agree for {}!'.format(varname)
        units_ref = refdata[varname].units
        units_dev = devdata[varname].units
        assert units_ref == units_dev, 'Units do not match for {}!'.format(varname)
        
        # Set plot extent
        extent=(-90,90,0,nlev)
        
        # if normalizing by area, adjust units to be per m2, and adjust title string
        units = units_ref
        varndim = varndim_ref
        subtitle_extra = ''
        
        # Slice the data
        ds_ref = refdata[varname].isel(time=itime)
        ds_dev = devdata[varname].isel(time=itime)

        # if normalizing by area, transform on the native grid and adjust units and subtitle string
        exclude_list = ['WetLossConvFrac','Prod_','Loss_']
        if normalize_by_area and not any(s in varname for s in exclude_list):
            ds_ref.values = ds_ref.values / refdata['AREAM2'].values[np.newaxis,:,:]
            ds_dev.values = ds_dev.values / devdata['AREAM2'].values[np.newaxis,:,:]
            units = '{} m-2'.format(units)
            subtitle_extra = ', Normalized by Area'
            
        # Calculate zonal mean of the regridded data
        zm_ref = ds_ref.mean(axis=2)
        zm_dev = ds_dev.mean(axis=2)
            
        # Get min and max for colorbar limits
        [vmin_ref, vmax_ref] = [zm_ref.min(), zm_ref.max()]
        [vmin_dev, vmax_dev] = [zm_dev.min(), zm_dev.max()]
        vmin_cmn = np.min([vmin_ref, vmin_dev])
        vmax_cmn = np.max([vmax_ref, vmax_dev])
        if match_cbar: [vmin, vmax] = [vmin_cmn, vmax_cmn]
        
        # Create 2x2 figure
        figs, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, figsize=[12,12], 
                                                      subplot_kw={'projection': crs.PlateCarree()})
        # Give the page a title
        offset = 0.96
        fontsize=25
        figs.suptitle('{}, Zonal Mean'.format(varname), fontsize=fontsize, y=offset)

        # Subplot 0: Ref
        if not match_cbar: [vmin, vmax] = [vmin_ref, vmax_ref]
        plot0 = ax0.imshow(zm_ref, cmap=WhGrYlRd, extent=extent, vmin=vmin, vmax=vmax)
        ax0.set_title('{} (Ref){}\n{}'.format(refstr, subtitle_extra, llres ))
        ax0.set_aspect('auto')
        ax0.set_xticks(xtick_positions)
        ax0.set_xticklabels(xticklabels)
        ax0.set_yticks(ytick_positions)
        ax0.set_yticklabels(yticklabels)
        cb = plt.colorbar(plot0, ax=ax0, orientation='horizontal', pad=0.10)
        if (vmax-vmin) < 0.001 or (vmax-vmin) > 1000:
            cb.locator = ticker.MaxNLocator(nbins=4)
            cb.update_ticks()
        cb.set_label(units)
        
        # Subplot 1: Dev
        if not match_cbar: [vmin, vmax] = [vmin_dev, vmax_dev]
        plot1 = ax1.imshow(zm_dev, cmap=WhGrYlRd, extent=extent, vmin=vmin, vmax=vmax)
        ax1.set_title('{} (Dev){}\n{}'.format(devstr, subtitle_extra, llres))
        ax1.set_aspect('auto')
        ax1.set_xticks(xtick_positions)
        ax1.set_xticklabels(xticklabels)
        ax1.set_yticks(ytick_positions)
        ax1.set_yticklabels(yticklabels)
        cb = plt.colorbar(plot1, ax=ax1, orientation='horizontal', pad=0.10)
        if (vmax-vmin) < 0.001 or (vmax-vmin) > 1000:
            cb.locator = ticker.MaxNLocator(nbins=4)
            cb.update_ticks()
        cb.set_label(units)
            
        # Subplot 2: Difference
        zm_absdiff = zm_dev - zm_ref
        diffabsmax = max([np.abs(zm_absdiff.min()), np.abs(zm_absdiff.max())])
        [vmin, vmax] = [-diffabsmax, diffabsmax]
        plot2 = ax2.imshow(zm_absdiff, cmap='RdBu_r', extent=extent, vmin=vmin, vmax=vmax)
        ax2.set_title('Difference\n(Dev - Ref)')
        ax2.set_aspect('auto')
        ax2.set_xticks(xtick_positions)
        ax2.set_xticklabels(xticklabels)
        ax2.set_yticks(ytick_positions)
        ax2.set_yticklabels(yticklabels)
        cb = plt.colorbar(plot2, ax=ax2, orientation='horizontal', pad=0.10)
        if (vmax-vmin) < 0.001 or (vmax-vmin) > 1000:
            cb.locator = ticker.MaxNLocator(nbins=4)
            cb.update_ticks()
        cb.set_label(units)
        
        # Subplot 3: Fractional Difference (restrict to +/-2)
        zm_fracdiff = (zm_dev - zm_ref) / zm_ref
        if full_ratio_range: [vmin, vmax] = [None, None]
        else: [vmin, vmax] = [-2, 2]
        plot3 = ax3.imshow(zm_fracdiff, vmin=vmin, vmax=vmax, cmap='RdBu_r', extent=extent)
        ax3.set_title('Fractional Difference\n(Dev - Ref)/Ref')
        ax3.set_aspect('auto')
        ax3.set_xticks(xtick_positions)
        ax3.set_xticklabels(xticklabels)
        ax3.set_yticks(ytick_positions)
        ax3.set_yticklabels(yticklabels)
        cb = plt.colorbar(plot3, ax=ax3, orientation='horizontal', pad=0.10)
        cb.set_clim(vmin=vmin, vmax=vmax)
        cb.set_label('unitless')      
            
        if savepdf:    
            pdf.savefig(figs)
            plt.close(figs)
            
    if savepdf: pdf.close()

## Define directories and global information

In [10]:
# Shared high-level directory
testdir = '/n/home08/elundgren/GC/testruns/12.0.3'

# Ref and dev run output directories
refdir = os.path.join(testdir,'geosfp_4x5_standard')
devdir = os.path.join(testdir,'geosfp_4x5_standard')

# Ref and dev strings (for use in plots and messages)
refstr='12.0.3'
devstr='12.0.3'

# Set directory to store plots
plotsdir = os.path.join(testdir,'plots')

# Define cubed sphere resolutions (used for restart file comparison)
llres = '4x5'

# Check that paths exist
check_paths(refdir, devdir)

Path 1 exists: /n/home08/elundgren/GC/testruns/12.0.3/geosfp_4x5_standard
Path 2 exists: /n/home08/elundgren/GC/testruns/12.0.3/geosfp_4x5_standard


### Print Ref netcdf filenames

In [11]:
reffiles = [k for k in os.listdir(refdir) if '.nc' in k]
for k in reffiles:
    print(k)

GEOSChem.StateMet.20160701_0000z.nc4
HEMCO_diagnostics.201607010000.nc
GEOSChem.DryDep.20160701_0000z.nc4
GEOSChem.ConcAfterChem.20160701_0000z.nc4
GEOSChem.SpeciesConc.20160701_0000z.nc4
GEOSChem.AerosolMass.20160701_0000z.nc4
GEOSChem.JValues.20160701_0000z.nc4
GEOSChem.Aerosols.20160701_0000z.nc4
GEOSChem.CloudConvFlux.20160701_0000z.nc4
GEOSChem_restart.201608010000.nc
GEOSChem.WetLossConv.20160701_0000z.nc4
GEOSChem.WetLossLS.20160701_0000z.nc4
GEOSChem.StateChm.20160701_0000z.nc4
GEOSChem.ProdLoss.20160701_0000z.nc4
HEMCO_restart.201608010000.nc
GEOSChem.JValuesLocalNoon.20160701_0000z.nc4
GEOSChem.LevelEdgeDiags.20160701_0000z.nc4
GEOSChem_restart.201607010000.nc


### Print Dev netcdf filenames

In [12]:
devfiles = [k for k in os.listdir(devdir) if '.nc' in k]
for k in devfiles:
    print(k)

GEOSChem.StateMet.20160701_0000z.nc4
HEMCO_diagnostics.201607010000.nc
GEOSChem.DryDep.20160701_0000z.nc4
GEOSChem.ConcAfterChem.20160701_0000z.nc4
GEOSChem.SpeciesConc.20160701_0000z.nc4
GEOSChem.AerosolMass.20160701_0000z.nc4
GEOSChem.JValues.20160701_0000z.nc4
GEOSChem.Aerosols.20160701_0000z.nc4
GEOSChem.CloudConvFlux.20160701_0000z.nc4
GEOSChem_restart.201608010000.nc
GEOSChem.WetLossConv.20160701_0000z.nc4
GEOSChem.WetLossLS.20160701_0000z.nc4
GEOSChem.StateChm.20160701_0000z.nc4
GEOSChem.ProdLoss.20160701_0000z.nc4
HEMCO_restart.201608010000.nc
GEOSChem.JValuesLocalNoon.20160701_0000z.nc4
GEOSChem.LevelEdgeDiags.20160701_0000z.nc4
GEOSChem_restart.201607010000.nc


## Compare Restart Files

In [13]:
#reffile_rst = os.path.join(refdir,'GEOSChem_restart.201607010000.nc')
#devfile_rst = os.path.join(devdir,'GEOSChem_restart.201607010000.nc')
#check_paths(reffile_rst, devfile_rst)

In [14]:
#refdata_rst = xr.open_dataset(reffile_rst)
#devdata_rst = xr.open_dataset(devfile_rst)
#[commonvars, commonvars2D, commonvars3D] = compare_varnames(refdata_rst, refstr, devdata_rst, devstr)

In [15]:
#get_stats(refdata_rst, refstr, devdata_rst, devstr, 'SPC_CFC11')

In [16]:
#compare_zonal_mean( refdata_rst, refstr, devdata_rst, devstr, llres, varlist=['SPC_O3'] )

In [17]:
#compare_single_level( refdata_rst, refstr, devdata_rst, devstr, llres, varlist=['SPC_O3'] )

## Inspect time-averaged species concentration diagnostic

In [18]:
day = '20160701'
time = '0000'
collection = 'SpeciesConc'
[refdata, devdata] = get_collection_data(refdir, devdir, collection, day, time)

Path 1 exists: /n/home08/elundgren/GC/testruns/12.0.3/geosfp_4x5_standard/GEOSChem.SpeciesConc.20160701_0000z.nc4
Path 2 exists: /n/home08/elundgren/GC/testruns/12.0.3/geosfp_4x5_standard/GEOSChem.SpeciesConc.20160701_0000z.nc4


In [19]:
[commonvars, commonvars2D, commonvars3D] = compare_varnames(refdata, refstr, devdata, devstr)

168 common variables (0 are 2-dim and 162 are 3-dim)


In [20]:
CFCs=[k for k in commonvars if 'CFC' in k]
print(CFCs)

['SpeciesConc_CFC11', 'SpeciesConc_CFC113', 'SpeciesConc_CFC114', 'SpeciesConc_CFC115', 'SpeciesConc_CFC12', 'SpeciesConc_HCFC123', 'SpeciesConc_HCFC141b', 'SpeciesConc_HCFC142b', 'SpeciesConc_HCFC22']


In [21]:
varname='SpeciesConc_CFC11'
get_stats(refdata, refstr, devdata, devstr, varname)

Data units: mol mol-1 dry
Array sizes:
    12.0.3:  (1, 72, 46, 72)
    12.0.3:  (1, 72, 46, 72)
Global stats:
  Mean:
    12.0.3:  1.1334942012775073e-10
    12.0.3:  1.1334942012775073e-10
  Min:
    12.0.3:  0.0
    12.0.3:  0.0
  Max:
    12.0.3:  2.193199927225109e-10
    12.0.3:  2.193199927225109e-10
  Sum:
    12.0.3:  2.7029756893170998e-05
    12.0.3:  2.7029756893170998e-05


In [22]:
#compare_single_level( refdata, refstr, devdata, devstr, llres, varlist=[varname] )

In [23]:
#compare_zonal_mean( refdata, refstr, devdata, devstr, llres, varlist=[varname] )

## Create PDF of plots for list of species

In [24]:
# Define subset of variables in files
#varlist = commonvars3D
varlist = [varname]
#varlist = [k for k in commonvars3D if 'CFC' in k]

In [25]:
# Surface
#pdfname = os.path.join(plotsdir,'{}_Surface.pdf'.format(ref_collection))
pdfname = os.path.join(plotsdir,'{}_Surface.pdf'.format(varname))
#pdfname = os.path.join(plotsdir,'{}_Surface.pdf'.format('CFCs'))
compare_single_level( refdata, refstr, devdata, devstr, llres, varlist=varlist, ilev=0,
                     savepdf=True, pdfname=pdfname )


Creating /n/home08/elundgren/GC/testruns/12.0.3/plots/SpeciesConc_CFC11_Surface.pdf for 1 variables
0 

In [26]:
# Zonal mean
#pdfname = os.path.join(plotsdir,'{}_ZonalMean.pdf'.format(ref_collection))
pdfname = os.path.join(plotsdir,'{}_ZonalMean.pdf'.format(varname))
#pdfname = os.path.join(plotsdir,'{}_ZonalMean.pdf'.format('CFCs'))
compare_zonal_mean(refdata, refstr, devdata, devstr, llres, varlist=varlist, 
                   savepdf=True, pdfname=pdfname )


Creating /n/home08/elundgren/GC/testruns/12.0.3/plots/SpeciesConc_CFC11_ZonalMean.pdf for 1 variables
0 

## Other diagnostic collections

In [27]:
# Inspect the available files again, but only show those that match
commonfiles = [k for k in reffiles if k in devfiles]
commonfiles

['GEOSChem.StateMet.20160701_0000z.nc4',
 'HEMCO_diagnostics.201607010000.nc',
 'GEOSChem.DryDep.20160701_0000z.nc4',
 'GEOSChem.ConcAfterChem.20160701_0000z.nc4',
 'GEOSChem.SpeciesConc.20160701_0000z.nc4',
 'GEOSChem.AerosolMass.20160701_0000z.nc4',
 'GEOSChem.JValues.20160701_0000z.nc4',
 'GEOSChem.Aerosols.20160701_0000z.nc4',
 'GEOSChem.CloudConvFlux.20160701_0000z.nc4',
 'GEOSChem_restart.201608010000.nc',
 'GEOSChem.WetLossConv.20160701_0000z.nc4',
 'GEOSChem.WetLossLS.20160701_0000z.nc4',
 'GEOSChem.StateChm.20160701_0000z.nc4',
 'GEOSChem.ProdLoss.20160701_0000z.nc4',
 'HEMCO_restart.201608010000.nc',
 'GEOSChem.JValuesLocalNoon.20160701_0000z.nc4',
 'GEOSChem.LevelEdgeDiags.20160701_0000z.nc4',
 'GEOSChem_restart.201607010000.nc']

In [28]:
# Template for inspecting any collection:

# To specify file:
#day = {edit}
#time = {edit}
#collection = {edit}

# For retrieving the data:
#[refdata, devdata] = get_collection_data(refdir, devdir, collection, day, time)

# For inspecting interactively:
#refdata
#devdata
#[commonvars, commonvars2D, commonvars3D] = compare_varnames(refdata, refstr, devdata, devstr)
# varname = {edit}
#get_stats(refdata, refstr, devdata, devstr, varname)
# varlist = {edit}
#compare_single_level(refdata, refstr, devdata, devstr, llres, varlist=[varname] )
#compare_zonal_mean(refdata, refstr, devdata, devstr, llres, varlist=[varname] )

# For creating PDFs:
#prefix = {edit}
#pdfname = os.path.join(plotsdir,'{}_Surface.pdf'.format(prefix))
#compare_single_level(refdata, refstr, devdata, devstr, llres, varlist=varlist, ilev=0,
#                     savepdf=True, pdfname=pdfname )
#pdfname = os.path.join(plotsdir,'{}_ZonalMean.pdf'.format(prefix))
#compare_zonal_mean(refdata, refstr, devdata, devstr, varlist=varlist, llres, 
#                   savepdf=True, pdfname=pdfname )

# For other function options use shift-tab while hovering over the function name in a cell that calls it